In [1]:
import netCDF4 as nc
import numpy as np
import datetime as dt
import shutil
import os
import glob

In [7]:
#find bad files:
for el in sorted(glob.glob('/results/forcing/atmospheric/GEM2.5/gemlam/gemlam_y????m??d??.nc')):
    #print(el)
    with nc.Dataset(el) as f:
        tdiff=f.variables['time_counter'][-1]-f.variables['time_counter'][0]
    if tdiff>24*3600:
        origfile=el
        fname=el.split('/')[-1]
        destfile='/data/eolson/results/MEOPAR/GEMLAM/'+fname
        date=dt.datetime.strptime(fname,'gemlam_y%Ym%md%d.nc')
        print(origfile,destfile,date)

<ipython-input-7-728e0422ff8e>:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tdiff=f.variables['time_counter'][-1]-f.variables['time_counter'][0]


/results/forcing/atmospheric/GEM2.5/gemlam/gemlam_y2008m07d23.nc /data/eolson/results/MEOPAR/GEMLAM/gemlam_y2008m07d23.nc 2008-07-23 00:00:00
/results/forcing/atmospheric/GEM2.5/gemlam/gemlam_y2008m08d12.nc /data/eolson/results/MEOPAR/GEMLAM/gemlam_y2008m08d12.nc 2008-08-12 00:00:00
/results/forcing/atmospheric/GEM2.5/gemlam/gemlam_y2009m09d05.nc /data/eolson/results/MEOPAR/GEMLAM/gemlam_y2009m09d05.nc 2009-09-05 00:00:00
/results/forcing/atmospheric/GEM2.5/gemlam/gemlam_y2009m09d06.nc /data/eolson/results/MEOPAR/GEMLAM/gemlam_y2009m09d06.nc 2009-09-06 00:00:00
/results/forcing/atmospheric/GEM2.5/gemlam/gemlam_y2009m09d07.nc /data/eolson/results/MEOPAR/GEMLAM/gemlam_y2009m09d07.nc 2009-09-07 00:00:00
/results/forcing/atmospheric/GEM2.5/gemlam/gemlam_y2009m09d08.nc /data/eolson/results/MEOPAR/GEMLAM/gemlam_y2009m09d08.nc 2009-09-08 00:00:00
/results/forcing/atmospheric/GEM2.5/gemlam/gemlam_y2010m02d06.nc /data/eolson/results/MEOPAR/GEMLAM/gemlam_y2010m02d06.nc 2010-02-06 00:00:00
/resul

In [8]:
def fixfile(origfile,destfile,date0):
    
    if os.path.isfile(destfile):
        os.remove(destfile)

    fold=nc.Dataset(origfile,'r')

    #[dt.datetime(1950,1,1)+dt.timedelta(seconds=ii) for ii 
    #      in fold.variables['time_counter'][:]]

    fnew=nc.Dataset(destfile,'w')

    ttype=fold.variables['time_counter'].dtype

    newcounter=[(date0-dt.datetime(1950,1,1)).total_seconds()+
               3600*ii for ii in range(0,24)]

    fold.variables['time_counter'].dimensions

    for name, dimension in fold.dimensions.items():
        fnew.createDimension(name, (len(dimension) if not dimension.isunlimited() else None))

    fnew.createVariable('time_counter',ttype,('time_counter',),fill_value=fold['time_counter'].getncattr('_FillValue'),zlib=True)
    fnew['time_counter'][:]=newcounter[:]

    atlist=[el for el in fold['time_counter'].ncattrs() if el not in ('_FillValue',)]
    atlist

    for el in atlist:
        fnew['time_counter'].setncattr(el,fold['time_counter'].getncattr(el))

    for name, variable in fold.variables.items():
            if name not in ('time_counter',):
                x = fnew.createVariable(name, variable.datatype, variable.dimensions,
                                        fill_value=variable.getncattr('_FillValue'),zlib=True)
                fnew[name][:] = fold[name][:]
                atlist=[el for el in variable.ncattrs() if el not in ('_FillValue',)]
                for el in atlist:
                    fnew[name].setncattr(el,variable.getncattr(el))

    fnew.history_of_appended_files=fold.history_of_appended_files

    fnew.history=f'{dt.datetime.now()}: revised time_counter to accurately reflect day and time using Python netCDF4 \n'+fold.history

    fnew.NCO=fold.NCO

    fnew.nco_openmp_thread_number=fold.nco_openmp_thread_number

    fnew.close()
    fold.close()

In [10]:
#fix bad files:
for el in sorted(glob.glob('/results/forcing/atmospheric/GEM2.5/gemlam/gemlam_y????m??d??.nc')):
    #print(el)
    with nc.Dataset(el) as f:
        tdiff=f.variables['time_counter'][-1]-f.variables['time_counter'][0]
    if tdiff>24*3600:
        origfile=el
        fname=el.split('/')[-1]
        destfile='/data/eolson/results/MEOPAR/GEMLAM/'+fname
        date=dt.datetime.strptime(fname,'gemlam_y%Ym%md%d.nc')
        fixfile(origfile,destfile,date)
        print(date)

<ipython-input-10-72cdd5df59a9>:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tdiff=f.variables['time_counter'][-1]-f.variables['time_counter'][0]
<ipython-input-8-e05b92b2a657>:36: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fnew[name][:] = fold[name][:]


2008-07-23 00:00:00
2008-08-12 00:00:00
2009-09-05 00:00:00
2009-09-06 00:00:00
2009-09-07 00:00:00
2009-09-08 00:00:00
2010-02-06 00:00:00
2010-02-07 00:00:00
2010-03-31 00:00:00
2010-04-01 00:00:00
2010-10-12 00:00:00
2010-10-13 00:00:00
2010-10-14 00:00:00
2010-10-15 00:00:00
2011-07-17 00:00:00
2011-07-18 00:00:00


In [11]:
f=nc.Dataset(destfile)

In [12]:
f

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    history_of_appended_files: Wed Oct 23 14:32:32 2019: Appended file /dev/shm/precip.nc had following "history" attribute:
Wed Oct 23 14:32:31 2019: /usr/bin/ncdiff -4 -O -o /dev/shm/precip.nc -v precip /tmp/tmpym1xzb9n/gemlam_y2011m07d18_000.nc /tmp/tmpym1xzb9n/gemlam_y2011m07d17_023.nc
Wed Oct 23 14:28:46 2019: /usr/bin/ncatted -a missing_variables,global,d,, /tmp/tmpym1xzb9n/gemlam_y2011m07d18_000.nc
Wed Oct 23 14:28:46 2019: /usr/bin/ncks -4 -A -v solar /dev/shm/solar.nc /tmp/tmpym1xzb9n/gemlam_y2011m07d18_000.nc
Wed Oct 23 14:28:46 2019: /usr/bin/ncks -4 -O -x -v solar /tmp/tmpym1xzb9n/gemlam_y2011m07d18_000.nc /tmp/tmpym1xzb9n/gemlam_y2011m07d18_000.nc
Wed Oct 23 12:08:20 2019: /usr/bin/ncap2 -4 -O -s RT=RT*1000 /tmp/tmpym1xzb9n/2011071712_012.nc /tmp/tmpym1xzb9n/2011071712_012.nc
Wed Oct 23 12:08:20 2019: /usr/bin/ncap2 -4 -O -s PR=PR/3.6 /tmp/tmpym1xzb9n/2011071712_012.nc /tmp/tmpym1xzb9n/20

In [13]:
f.variables

{'time_counter': <class 'netCDF4._netCDF4.Variable'>
 float64 time_counter(time_counter)
     _FillValue: nan
     axis: T
     calendar: proleptic_gregorian
     cell_methods: time_counter: mean
     long_name: Time axis
     standard_name: time
     time_origin: 1950-01-01 00:00:00
     title: Time
     units: seconds since 1950-01-01
 unlimited dimensions: time_counter
 current shape = (24,)
 filling on,
 'PRATE_surface': <class 'netCDF4._netCDF4.Variable'>
 float32 PRATE_surface(time_counter, y, x)
     _FillValue: nan
     level: surface
     long_name: Precipitation Rate
     standard_name: precipitation_flux
     units: kg/m^2/s
 unlimited dimensions: time_counter
 current shape = (24, 266, 256)
 filling on,
 'RH_2maboveground': <class 'netCDF4._netCDF4.Variable'>
 float32 RH_2maboveground(time_counter, y, x)
     _FillValue: nan
     level: 2 m above surface
     long_name: Relative Humidity
     standard_name: relative_humidity_2maboveground
     units: percent
     comment: c

In [14]:
[dt.datetime(1950,1,1)+dt.timedelta(seconds=ii) for ii 
      in f.variables['time_counter'][:]]

<ipython-input-14-6bbf72d9b8e1>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  in f.variables['time_counter'][:]]


[datetime.datetime(2011, 7, 18, 0, 0),
 datetime.datetime(2011, 7, 18, 1, 0),
 datetime.datetime(2011, 7, 18, 2, 0),
 datetime.datetime(2011, 7, 18, 3, 0),
 datetime.datetime(2011, 7, 18, 4, 0),
 datetime.datetime(2011, 7, 18, 5, 0),
 datetime.datetime(2011, 7, 18, 6, 0),
 datetime.datetime(2011, 7, 18, 7, 0),
 datetime.datetime(2011, 7, 18, 8, 0),
 datetime.datetime(2011, 7, 18, 9, 0),
 datetime.datetime(2011, 7, 18, 10, 0),
 datetime.datetime(2011, 7, 18, 11, 0),
 datetime.datetime(2011, 7, 18, 12, 0),
 datetime.datetime(2011, 7, 18, 13, 0),
 datetime.datetime(2011, 7, 18, 14, 0),
 datetime.datetime(2011, 7, 18, 15, 0),
 datetime.datetime(2011, 7, 18, 16, 0),
 datetime.datetime(2011, 7, 18, 17, 0),
 datetime.datetime(2011, 7, 18, 18, 0),
 datetime.datetime(2011, 7, 18, 19, 0),
 datetime.datetime(2011, 7, 18, 20, 0),
 datetime.datetime(2011, 7, 18, 21, 0),
 datetime.datetime(2011, 7, 18, 22, 0),
 datetime.datetime(2011, 7, 18, 23, 0)]

In [15]:
f.close()

In [16]:
import xarray as xr

In [19]:
flist=['/data/eolson/results/MEOPAR/GEMLAM/gemlam_y2010m03d30.nc',
       '/data/eolson/results/MEOPAR/GEMLAM/gemlam_y2010m03d31.nc',
       '/data/eolson/results/MEOPAR/GEMLAM/gemlam_y2010m04d01.nc',
       '/data/eolson/results/MEOPAR/GEMLAM/gemlam_y2010m04d02.nc']

In [20]:
test=xr.open_mfdataset(flist)

In [21]:
test.close

<bound method DataWithCoords.close of <xarray.Dataset>
Dimensions:           (time_counter: 96, x: 256, y: 266)
Coordinates:
  * time_counter      (time_counter) datetime64[ns] 2010-03-30 ... 2010-04-02...
Dimensions without coordinates: x, y
Data variables:
    nav_lon           (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    nav_lat           (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    qair              (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    RH_2maboveground  (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    therm_rad         (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    u_wind            (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    v_wind            (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    at